### import
- 기준일 다음날 정식,약식데이터 다운 > 약식 업로드 > 정식 양식 업로드 결과 (목록을 최대한 비슷하게 하기 위해 정식도 기준일 다음날 받는다.)
- 컷오프요일이 언제든 금액정보는 정확히 반영할 수 있음
- 유의사항
    - 진행상태가 실효라도 실효일이 없으면 실효되지 않을 수도 있다.(미납회수가 실효기준을 충족하고 입금예정일(실효예정일)에 입금이 없는 경우. 요일상관없이 3일뒤 실효확정되고, 그날 오후(아마 입금업무마간시간후겠지)에 실효일 찍히고 그 다음날 실효통지. 그런데 그전에 입금해버리면 실효안됨. 진행상태는 실시간이므로 약식통합이든, 정식만으로 하든 동일한 결과). 실효가 확정되기 전까지는 실효일자 없음. 따라서 실효일자 없는 실효는 컷오프일 기준으로는 엄밀히 말해 실효가 아니라 실효예정이다. -> 정상으로 변경
        - 완제일 없는 중도완제는 입금처리가 안 된 것일뿐 오후 되면 그날로 날짜 박힘.(중도완제 맞음)
    - 막 채권신고한 경우, 약식에서는 계좌번호 나오나, 정식에서는 접수통지중으로 나오는 경우, 약식을 기준으로 병합하므로 진행중으로 처리된다. 대신 정식에만 있는 칼럼은 모두 빈값

#### 변경로그
- 실효예정 : [포함,실효but실효일없음] > 정상이행으로 240823

In [1]:
import os, re, pickle
import pandas as pd
import numpy as np
from os.path import join
from tqdm import tqdm
import warnings
warnings.simplefilter("ignore")
import functions
from datetime import datetime, timedelta
import calendar
from dateutil.relativedelta import relativedelta
from pandas.tseries.offsets import DateOffset

# dtype 정의 (사이트 파일은 읽지 말자. 할 게 너무 많음)
# 나우리 타입 반영
nauri_dtype = {"채무자키":str, "계좌키":str, '보증인키':str, '신용회복키' : str, "계좌번호":str, "심의차수":str, "변제금수취계좌":str}
account_dtype = {'채무자키':str, '계좌키':str, '타채무자키':str}

### 파일 읽기

In [3]:
# 계좌별 진행상황 파일 읽기
path_base = r"D:\3.자산\신용회복\신용회복 전체데이터\계좌별 진행상황"
########################################################
약식병합 = False ############# 금요일(일요일)이 컷오프가 아니라면 약식병합해야-------
basedate = "250525" # 정식 업로드결과 파일 날짜 컷오프일에 맞춰 수정
company = "솔림"  # 솔림 or 대성
basedate_계좌 = basedate # "yymmdd" 또는 basedate ######################## 계좌조회새창 받기 전이면 전달로
########################################################

# 읽을 파일명
suffix = "통합" if company!="대성" else "대성"

filename = basedate + "_계좌별 진행상황 조회_" + suffix

filename_정식 = filename + "_업로드결과"
filename_약식 = filename + "_약식_업로드결과"

# 계좌별 진행상황 조회 파일 읽기 및 pkl 저장
    # 정식
정식_ori = pd.read_pickle(join(path_base, "pkl", filename_정식+".pkl" ))
    # 약식
if 약식병합 : 
    약식_ori = pd.read_pickle(join(path_base, "pkl", filename_약식+".pkl" ))

In [4]:
# 계좌조회새창 읽기 (메모 : 통합신고건 처리위해) --------------
dir_account = join(r"D:\3.자산\전산 dataset", company, basedate_계좌, "pkl")
account_ori = pd.read_pickle(functions.키워드로파일명찾기(dir_account, "계좌조회새창", 전체경로=True))
# # 계좌 엑셀로 읽을 때
# excel_path = functions.키워드로파일명찾기(join(r"D:\3.자산\전산 dataset", company, basedate_계좌), "계좌조회새창", 전체경로=True)
# account_ori = pd.read_excel(excel_path, dtype=account_dtype).fillna("")

# 보증인조회새창 읽기 (키 매칭 확인)
grt = pd.read_pickle(functions.키워드로파일명찾기(dir_account, "보증인새창", "새보증인", 전체경로=True))

### 전처리

##### 키 중복건 처리

In [5]:
# 재시작 위해 복사본 만들기
if 약식병합 : 
    약식 = 약식_ori.copy()
    print("약식건수", len(약식))
    # 키값에 중복건 있는지 확인, merge때문에 약식, 정식 모두 해야함
    약식["pk"] = 약식.신청인명 + 약식.신청인주민번호 + 약식.접수번호 + 약식.계좌번호
    print("약식 키 중복건")
    display(약식[약식.pk.duplicated(keep=False)])

정식 = 정식_ori.copy()
print("정식건수", len(정식))

정식["pk"] = 정식.신청인명 + 정식.신청인주민번호 + 정식.접수번호 + 정식.계좌번호
print("정식 키 중복건")
display(정식[정식.pk.duplicated(keep=False)])

account = account_ori[["계좌키","채무자키","메모"]].copy()

정식건수 17195
정식 키 중복건


,채무자키,계좌키,보증인키,신용회복키,채무자명,채무상태,현재채권구분,신청인명,신청인주민번호,접수번호,...,등록파일,수정파일,삭제여부,등록일자,등록유저,수정일자,수정유저,확인결과,저장결과,pk


In [6]:
# 둘 중 하나 작업 ####################################################################
# 1) 키값 입력(채권신고전) - 키값은 중복 제거 후 아래 채무자키 계좌키 없는 거에서 해도 됨, 채권신고전은 채권신고전으로 됨
# 정식.loc[[795,14729],["채무자키","계좌키"]] = [["20463762", "200983812"], ["20463762", "200983812"]]


# 2) 중복건 삭제(ex솔림-신탁 중복건)
정식_삭제전 = len(정식)
정식 = 정식.drop_duplicates(subset=["pk"], keep='first', inplace=False, ignore_index=True)
정식_삭제후 = len(정식)
if 약식병합 : 
    약식_삭제전 = len(약식)
    약식 = 약식.drop_duplicates(subset=["pk"], keep='first', inplace=False, ignore_index=True)
    약식_삭제후 = len(약식)
    print(f'약식 삭제건수 : {약식_삭제전} - {약식_삭제후} = {약식_삭제전-약식_삭제후}')
print(f'정식 삭제건수 : {정식_삭제전} - {정식_삭제후} = {정식_삭제전-정식_삭제후}')

정식 삭제건수 : 17195 - 17195 = 0


In [7]:
# 포함 & 실효 but 실효일 없음 -> 정상이행으로(실효예정일뿐)
# (기준일 오후 5:30 이후에 받은 자료일 때만 정확.)
if 약식병합 : 
    실효임박index = 약식[(약식["계좌진행상태내용"]=="채무조정포함")&(약식["신청인진행상태"]=="중도실효")&(약식["실효/완제/합의서포기일자"]=="")].index
    약식.loc[실효임박index, "신청인진행상태"] = "정상이행"
else : 
    실효임박index = 정식[(정식["계좌진행상태내용"]=="채무조정포함")&(정식["신청인진행상태"]=="중도실효")&(정식["실효/완제/합의서포기일자"]=="")].index
    정식.loc[실효임박index, "신청인진행상태"] = "정상이행"

In [8]:
if 약식병합 : 
        # 정식 : 불필요 칼럼 삭제
        키열 = ["신청인명", "신청인주민번호", "접수번호", "계좌번호"]

        나우리칼럼불요 = ["채무자명", "채무상태", "현재채권구분","신청인진행상태변경전", "특별상환유예", "pk", # pk 추가
                "변경일", "변경구분", "신용회복진행상태","신복아이디","등록파일","수정파일","삭제여부","등록일자","등록유저","수정일자","수정유저","확인결과","저장결과"]
        나우리칼럼필요 = ["채무자키", "계좌키", "보증인키", "신용회복키"]
        약식열 = ["거래일","심의차수","신청인진행상태","계좌진행상태내용","실효/완제/합의서포기일자","접수통지일","접수일","점포명","확정일","합의서체결일","상환개시일",
                "최초대출원금","조정전원금","조정전이자","조정전연체이자","조정전비용","조정전합계","조정후원금","조정후이자","조정후연체이자","조정후비용","조정후합계",
                "원금감면여부","조정대상포함여부","총상환기간","유예기간","원금균등상환기간","원리균등상환기간","이자상환기간","납입회차","연체기간","총납입금액","총납입원금",
                "총납입이자","총납입기타채무","상환후잔액","감면율","급여가압류원금","채무구분","주채무자주민번호","주채무자성명","동의요청일","동의확정일","재조정여부","재조정횟수",
                "수정조정여부","수정조정횟수","재조정접수일자","재조정처리일자","재조정반송일자","실효취소요청일자","실효원상회복일자","상각구분","신청구분","일시납감면율",
                "차상위계층여부","미납발생일","부동산담보대출여부","거치기간","감면방식","상환방식","담보권실행유예기간","특별면책","재난상환유예","특례지원","이자율재조정","거치이자납입유예"]

        정식.drop(columns= 나우리칼럼불요 + 나우리칼럼필요 + 약식열, inplace=True, axis=1)

        # 약식 : 신복 약식에 있는 칼럼만 남기기
        약식 = 약식.loc[:, 나우리칼럼필요 + 키열 + 약식열]
        
        # merge : 약식을 기준으로 정식에만 있는 칼럼 merge(약식에도 있는 칼럼은 drop했으므로 그대로 붙이면 됨)
        raw = pd.merge(약식, 정식, on=["신청인명", "신청인주민번호", "접수번호", "계좌번호"], how='left')
        print(len(raw), ", 병합전 약식 개수 : ", len(약식))
        
else : 
        raw = 정식

In [9]:
# 신용회복 자료와 칼럼 순서 맞추기 : 재난상환유예를 특별면책 뒤, 특례지원 앞으로
cols = ["채무자키","계좌키","보증인키","신용회복키","신청인명","신청인주민번호","접수번호","거래일","계좌번호","심의차수","신청인진행상태","계좌진행상태내용","실효/완제/합의서포기일자",
        "접수통지일","접수일","점포명","확정일","합의서체결일","상환개시일","대출과목","조정전이율","조정후이율","최초대출원금","조정전원금","조정전이자","조정전연체이자","조정전비용",
        "조정전합계","조정후원금","조정후이자","조정후연체이자","조정후비용","조정후합계","원금감면여부","조정대상포함여부","총상환기간","유예기간","원금균등상환기간","원리균등상환기간",
        "이자상환기간","납입회차","연체기간","총납입금액","원금균등시작회차","원금균등종료회차","원리균등시작회차","원리균등종료회차","이자상환시작회차","이자상환종료회차","원금균등채무액",
        "총납입원금","총납입이자","총납입기타채무","상환후잔액","감면율","추심원금","급여가압류원금","채무구분","주채무자주민번호","주채무자성명","시작회차1","종료회차1","채무조정1적용이율1",
        "채무조정2적용이율1","채무조정1월상환액1","채무조정2월상환액1","시작회차2","종료회차2","채무조정1적용이율2","채무조정2적용이율2","채무조정1월상환액2","채무조정2월상환액2","시작회차3",
        "종료회차3","채무조정1적용이율3","채무조정2적용이율3","채무조정1월상환액3","채무조정2월상환액3","시작회차4","종료회차4","채무조정1적용이율4","채무조정2적용이율4","채무조정1월상환액4",
        "채무조정2월상환액4","시작회차5","종료회차5","채무조정1적용이율5","채무조정2적용이율5","채무조정1월상환액5","채무조정2월상환액5","시작회차6","종료회차6","채무조정1적용이율6","채무조정2적용이율6",
        "채무조정1월상환액6","채무조정2월상환액6","시작회차7","종료회차7","채무조정1적용이율7","채무조정2적용이율7","채무조정1월상환액7","채무조정2월상환액7","시작회차8","종료회차8","채무조정1적용이율8",
        "채무조정2적용이율8","채무조정1월상환액8","채무조정2월상환액8","시작회차9","종료회차9","채무조정1적용이율9","채무조정2적용이율9","채무조정1월상환액9","채무조정2월상환액9","시작회차10","종료회차10",
        "채무조정1적용이율10","채무조정2적용이율10","채무조정1월상환액10","채무조정2월상환액10","동의요청일","동의확정일","재조정여부","재조정횟수","수정조정여부","수정조정횟수","재조정이전납입금액",
        "원금완제면제이자","출금정지등록여부","재조정접수일자","재조정처리일자","재조정반송일자","실효취소요청일자","실효원상회복일자","상각구분","신청구분","일시납감면율","차상위계층여부","미납발생일",
        "부동산담보대출여부","거치기간","감면방식","상환방식","체증식계좌여부","체증전구간의시작회차","체증전구간의종료회차","체증전구간의기준기간","변제금수취계좌","담보권실행유예기간","특별면책",
        "재난상환유예","특례지원","이자율재조정","거치이자납입유예"
]
raw = raw.loc[:,cols]


### 반영OPB, 진행구분, 상환구분

In [10]:
# 새로운 열을 추가 : insert 여러번 쓰니 성능 경고나와서
진행구분 = pd.Series([None] * len(raw), name='진행구분')
반영OPB = pd.Series([None] * len(raw), name='반영OPB')
상환구분 = pd.Series([None] * len(raw), name='상환구분')

position = 4
raw = pd.concat([raw.iloc[:, :position], 진행구분, 반영OPB, 상환구분, raw.iloc[:, position:]], axis=1)

# 전산미등록건 처리 #####################################################
if company == "통합" : 
    미등록주민번호 = ['771027-2221231'] #강민경
    cond = raw['신청인주민번호'].isin(미등록주민번호)
    raw.loc[cond, '채무자키'] = '전산미등록건'
    raw.loc[cond, '계좌키'] = '전산미등록건'
else :  # 대성
    미등록주민번호 = ['741008-1037838'] #강승욱
    cond = raw['신청인주민번호'].isin(미등록주민번호)
    raw.loc[cond, '채무자키'] = '전산미등록건'
    raw.loc[cond, '계좌키'] = '전산미등록건'
########################################################################

In [11]:
# 주민 계좌 중복건 : 전산에는 중복건 있더라도 신복위에서 받은 데이터에는 없는것이 당연(솔림,신탁 동시 채권신고요청시에는 있을 수 있음)
raw[raw.duplicated(subset=["신청인주민번호", "계좌번호"], keep=False)]

,채무자키,계좌키,보증인키,신용회복키,진행구분,반영OPB,상환구분,신청인명,신청인주민번호,접수번호,...,체증전구간의시작회차,체증전구간의종료회차,체증전구간의기준기간,변제금수취계좌,담보권실행유예기간,특별면책,재난상환유예,특례지원,이자율재조정,거치이자납입유예


In [12]:
# 진행구분열 작성-------------------------------
x = raw.신청인진행상태
y = raw.계좌진행상태내용
z = raw.계좌번호
r = raw.재조정여부

conds = [
    (x == '정상이행') & (y == '채무조정포함'),
    (y == "채무조정제외"),
    (x.str.contains(r"완제|상환") | y.str.contains(r"완제|상환") | z.str.contains(r"완제|상환")),
    (z == "채권신고전"),
    (x.str.contains(r"심사반송|중도실효|합의서체결포기")),
    (r == "Y")  ######## 누락채권포함재조정의 경우도 확정으로 되어버리는 문제가 있으나, 건수별로 없고 실제로 의결권 과반 넘지 않아 확정되는거나 마찬가지이므로 무시
]
values = ["확정", "채무조정제외","완제","채권신고전","실효","확정"]
default_value = "미확정"

raw["진행구분"] = np.select(conds, values, default_value)

# 반영OPB - 확정미확정 건-------------------------------
x = raw.진행구분
# 확정시 우선순위  
# 정상건인데 상환후잔액이 없는 경우는, 실효취소 및 재조정합의 후 첫회차 입금이 안 된 경우
# 조정후비용이나 이자가 있는 경우가 있기 때문에 원래 2순위로 조정후 합계가 반영되어야 하나, 회계법인 평가법에 아래와 같이 나와 있고, 이를 다른 회사도 적용하고 있다고 해서 그냥 이대로 적용하기로.
# 또한 약식을 활용하면 상환후잔액이 없는 경우가 거의 없어서 큰 문제는 없음.
a = raw.상환후잔액
b = raw.원금균등채무액
c= raw.조정후원금
d = raw.조정전원금 + raw.조정전비용   # 조정정비용 추가 2024-04-03

# 미확정-조정전원금0 은 이자채권으로 그냥 0이 맞다.
conds = [(x == "미확정"), (x =="확정")&(a!=0), (x =="확정")&(b!=0), (x =="확정")&(c!=0), (x =="확정")&(d!=0)]

# 조건에 따라 값을 설정
values = [d, a, b, c, d]

# 기본값 설정
default_value = np.nan

# 반영OPB 열 설정
raw["반영OPB"] = np.select(conds, values, default_value)


# 상환구분 작성 - 확정미확정 건-------------------------------
x = raw.상환방식

conds = [x.str.contains("원금균등"), x.str.contains("원리금균등")] #체증식(원금균등)이란게 있어서 contains로 바꿈

# 조건에 따라 값을 설정
values = ["개인","프리"]

# 기본값 설정
default_value = "" # 미확정

# 상환구분 열 설정
raw["상환구분"] = np.select(conds, values, default_value)

### 키매칭 오류 체크

In [13]:
# 정상건 중 키 매칭 오류건
정상조건 = raw.진행구분.str.contains('확정')
정상차주 = raw[정상조건 & (raw.채무구분=="주채무")]
정상보증인 = raw[정상조건 & (raw.채무구분=="보증채무")]

print('차주인데 계좌키 중복건')
display(정상차주[정상차주.계좌키.duplicated(keep=False)])
print('보증인키 중복건')
display(정상보증인[정상보증인.보증인키.duplicated(keep=False)])
print('삭제된 보증인키 매칭')
display(정상보증인[~정상보증인.보증인키.isin(grt["보증인키"])])

차주인데 계좌키 중복건


,채무자키,계좌키,보증인키,신용회복키,진행구분,반영OPB,상환구분,신청인명,신청인주민번호,접수번호,...,체증전구간의시작회차,체증전구간의종료회차,체증전구간의기준기간,변제금수취계좌,담보권실행유예기간,특별면책,재난상환유예,특례지원,이자율재조정,거치이자납입유예


보증인키 중복건


,채무자키,계좌키,보증인키,신용회복키,진행구분,반영OPB,상환구분,신청인명,신청인주민번호,접수번호,...,체증전구간의시작회차,체증전구간의종료회차,체증전구간의기준기간,변제금수취계좌,담보권실행유예기간,특별면책,재난상환유예,특례지원,이자율재조정,거치이자납입유예


삭제된 보증인키 매칭


,채무자키,계좌키,보증인키,신용회복키,진행구분,반영OPB,상환구분,신청인명,신청인주민번호,접수번호,...,체증전구간의시작회차,체증전구간의종료회차,체증전구간의기준기간,변제금수취계좌,담보권실행유예기간,특별면책,재난상환유예,특례지원,이자율재조정,거치이자납입유예


In [ ]:
# # 수정 : 나우리 키매칭 오류나던건 나우리에서 수정처리 완료
# raw.loc[71, "보증인키"] = "200189959" # 이종민 계좌키 200938533인데 보증인키 ~58인 경우
# raw.loc[1833, "보증인키"] = "200200624" # 임동환 보증인키 200185128로 나오는 경우

### 시트작성 : 최종,정상,제외

In [ ]:
# 반영OPB int로
raw["반영OPB"] = raw["반영OPB"].astype('Int64') # Int64는 int64와 달리 na도 허용해준다. 대신 확정미확정건에서 na있는지 확인해야함

# 최종, 확정미확정, 일부계좌제외 시트
최종 = raw.query('진행구분 in ["확정","미확정","실효","완제"]').copy()
확정미확정 = raw.query('진행구분 in ["확정", "미확정"]').copy()
일부계좌제외 = raw.query('진행구분 == "채무조정제외" and 채무자키 != ""') # 계좌번호 변경한 건들 있어서 채무자키 없을 수 있음, 20240117 기준 솔림 8건
# 일부계좌제외는 자산정리 채무상태에서 신복으로 처리하여 오류탐지 제외 ~~

# 전산미등록건, 일부계좌제외
최종.drop(최종[최종.채무자키=='전산미등록건'].index, inplace=True)
확정미확정.drop(확정미확정[확정미확정.채무자키=='전산미등록건'].index, inplace=True)
일부계좌제외.drop(일부계좌제외[일부계좌제외.채무자키=='전산미등록건'].index, inplace=True)

# 포함과 제외가 공존하는 제외건만 : 제외자 주민번호가 확정미확정에도 있으면
일부계좌제외 = 일부계좌제외.merge(확정미확정["신청인주민번호"], how="inner", on=["신청인주민번호"])

# 순번열 넣기
if "순번" not in raw.columns : 
    raw.insert(0, "순번", range(1,len(raw)+1))
    최종.insert(0, "순번", range(1,len(최종)+1))
    확정미확정.insert(0, "순번", range(1,len(확정미확정)+1))
    일부계좌제외.insert(0, "순번", range(1,len(일부계좌제외)+1))
    
# 확정미확정 반영opb에 na값 있는지 확인
print('확정미확정 반영opb에 na값 있는지 확인')
display(확정미확정[확정미확정['반영OPB'].isna()])

In [ ]:
# 체크
# 반영OPB
반영OPB오류 = raw[(raw.반영OPB == np.nan)|(raw.반영OPB == 0)|(raw.반영OPB=="")]
cols = ["채무자키", "계좌키", "진행구분", "상환구분","반영OPB", "조정전원금"]
키cols = ["채무자키", "계좌키", "보증인키", "신용회복키", "진행구분", "반영OPB", "상환구분", "신청인명", "신청인주민번호", "신청인진행상태", "계좌진행상태내용", "채무구분"]
if len(반영OPB오류) > 0 : 
    print('반영OPB 0원 ------')
    print('원금,법비용 0원계좌라도 상환예정액 조금이라도 나옴. 미확정건이면 전산opb를 따르니 0원 되겠지') ######################
    display(반영OPB오류[cols])
else : print("반영OPB 오류 없음")

# 확정미확정에 신용회복키 중복여부
신용회복키중복 = 확정미확정[확정미확정.duplicated('신용회복키', keep=False)]
if len(신용회복키중복) > 0 : 
    print('신용회복키 중복건 ------ 신용회복키 빈값 없애려면 업로드 후에 재다운로드해야')
    display(신용회복키중복[키cols])
else : print("확정미확정에 신용회복키 중복 없음")

# 채무자키, 계좌키, 신용회복키 매칭 오류
최종키오류 = 최종.query('채무자키=="" or 계좌키=="" or 신용회복키==""')
확정미확정키오류 = 확정미확정.query('채무자키=="" or 계좌키==""') # 신용회복키 없는 건 제외함 #or 신용회복키==""
print(f"전체 : {len(raw)}, 최종 : {len(최종)}, 확정미확정 : {len(확정미확정)}")
print(f"최종에서 키 없는 행 수 : {len(최종키오류)}, 확정미확정에서 키없는 행 수 : {len(확정미확정키오류)}")
if len(확정미확정키오류)>0 : 
    print('확정미확정 채무자키, 계좌키, 신복키 중 하나라도 없는 경우 ------')
    display(확정미확정키오류[키cols])
else : print("확정미확정 건에 키매칭 실패 없음")

# 확정 건 중 상환구분 없는 값
print("확정 건 중 상환구분 없는 값 :", len(확정미확정.query('상환방식=="확정" and 상환구분==""')))

# 일부계좌제외에 보증채무있는지
print("일부계좌제외에 보증채무수 :", len(일부계좌제외.query('채무구분=="보증채무"'))) # 제외이므로 확정미확정에는 없음

반영OPB 0원 ------
원금,법비용 0원계좌라도 상환예정액 조금이라도 나옴. 미확정건이면 전산opb를 따르니 0원 되겠지


,채무자키,계좌키,진행구분,상환구분,반영OPB,조정전원금
2875,20451902,200974602,미확정,개인,0,0
6189,20446361,200968077,미확정,개인,0,0


신용회복키 중복건 ------ 신용회복키 빈값 없애려면 업로드 후에 재다운로드해야


,채무자키,계좌키,보증인키,신용회복키,진행구분,반영OPB,상환구분,신청인명,신청인주민번호,신청인진행상태,계좌진행상태내용,채무구분
2349,20418419,200932290,,,미확정,682233,,이천기,590811-1382418,접수통지중,채무조정포함,주채무
2843,20514464,201045317,,,미확정,350567,,정진호,831217-1639516,접수통지중,채무조정포함,주채무
3336,20408879,200945798,,,미확정,1398160,,박정귀,940705-1251618,접수통지중,채무조정포함,주채무
9618,20418069,200931676,,,확정,2061858,개인,서혜정,610406-2173514,정상이행,채무조정포함,주채무
9619,20418069,200931677,,,확정,273085,개인,서혜정,610406-2173514,정상이행,채무조정포함,주채무
9620,20418439,200932311,,,확정,2486814,개인,서혜정,610406-2173514,정상이행,채무조정포함,주채무
10395,20410418,200975315,,,미확정,490000,,강유리,880103-2105511,접수통지중,채무조정포함,주채무
10815,20423825,200942178,,,미확정,9033610,,김광호,830821-1396816,접수통지중,채무조정포함,주채무
10861,20462354,200982404,200192245,,미확정,2045614,,김민석,771027-1907218,접수통지중,채무조정포함,보증채무
10877,20499214,201026901,,,미확정,10018834,,이해석,690126-1036616,접수통지중,채무조정포함,주채무


전체 : 17195, 최종 : 16908, 확정미확정 : 11653
최종에서 키 없는 행 수 : 99, 확정미확정에서 키없는 행 수 : 0
확정미확정 건에 키매칭 실패 없음
확정 건 중 상환구분 없는 값 : 0
일부계좌제외에 보증채무수 : 1


In [ ]:
# # # 키값 없는 거 수정
# 확정미확정.loc[4256,["채무자키","계좌키"]] = ["20408056","200922045"]
# 확정미확정.loc[10274,["채무자키","계좌키"]] = ["20412894","200919853"]

### 통합신고건 처리하기

In [26]:
통합신고계좌 = account.query("메모.str.contains('신용회복 통합신고')")
print("통합신고계좌수 :",len(통합신고계좌))
print("추가전 확정미확정 건수 :", len(확정미확정))

cnt = 확정미확정.순번.iloc[-1]

for i, v in 통합신고계좌.iterrows() : 
    피통합계좌 = re.search(r"[0-9]{9,9}", v.메모)[0]
    temp = 확정미확정.query('계좌키 == @피통합계좌').copy()
    if len(temp) == 1 :
        cnt += 1
        temp['순번'] = cnt
        temp['채무자키'] = v.채무자키
        temp['계좌키'] = v.계좌키
        temp[['신용회복키','계좌진행상태내용']] = '통합신고 ' + v.계좌키
        temp['반영OPB'] = 0
        
        # 행추가
        확정미확정 = pd.concat([확정미확정, temp])
    elif len(temp) == 0 :
        print("통합신고 취소, 원상회복 분할적용 확인 : ", v.계좌키, v.채무자키)
    
    else : 
        print("중복계좌키 발견 : ", v.계좌키, v.채무자키)
        
print("추가후 확정미확정 건수 :", len(확정미확정))

전체요약 = 확정미확정.groupby('진행구분').agg({'신용회복키':'count', '반영OPB':'sum'})
전체요약.loc["합계", :] = 전체요약.sum()
display(전체요약.applymap('{:,.0f}'.format))

통합신고계좌수 : 1
추가전 확정미확정 건수 : 11653
추가후 확정미확정 건수 : 11654


,신용회복키,반영OPB
진행구분,,
미확정,117,"568,324,842"
확정,"11,537","38,296,852,914"
합계,"11,654","38,865,177,756"


### 파일 저장

In [ ]:
# 저장
    # 확정미확정의 반영opb는 int64로
확정미확정["반영OPB"] = 확정미확정["반영OPB"].astype("int64")


# 기준데이터 저장 경로
if 약식병합 : 
    filename_write = basedate+"_신용회복_기준데이터_약식병합_"+ ("대성" if company=="대성" else "솔림")
else : 
    filename_write = basedate+"_신용회복_기준데이터_"+ ("대성" if company=="대성" else "솔림")
    
fullpath_result = join(os.path.dirname(path_base),"기준데이터", filename_write +".xlsx")


# 엑셀 파일로 저장
if not os.path.exists(fullpath_result):
    param = {'mode' : 'w'}
else : 
    param = {'mode' : 'a', 'if_sheet_exists' : 'replace'}

with pd.ExcelWriter(fullpath_result, engine='openpyxl', **param) as writer :
    raw.to_excel(writer, sheet_name='원본', index=False)
    최종.to_excel(writer, sheet_name='최종', index=False)
    확정미확정.to_excel(writer, sheet_name='확정,미확정', index=False)
    일부계좌제외.to_excel(writer, sheet_name='일부계좌제외', index=False)



# pkl 저장
확정미확정.to_pickle(join(os.path.dirname(path_base),"기준데이터", filename_write +".pkl"))

# [상태및담당자 체크용]
신복전체 = raw.query('채무자키!=""').copy() # 채무자키 없는 건은 다 종결건,중복건 등으로 문제없음을 확인. 채권신고전도 몇건 안되니 무시하자
    # 이름키 작성
신복전체['이름키_'] = 신복전체["채무자키"] + 신복전체["신청인명"]
    # 접수일 역정렬 후 중복제거
신복전체['접수일'] = pd.to_datetime(신복전체['접수일'])
신복전체 = 신복전체.sort_values(by='접수일', ascending=False)    
신복전체 = 신복전체.drop_duplicates(subset='이름키_')

    # pkl 저장
신복전체.to_pickle(join(os.path.dirname(path_base),"기준데이터", "[상태DC]신용회복_"+basedate+".pkl")) # pkl 로만 저장

### 실효처리 요청건 뽑기

#### 기본 및 파일읽기

In [28]:
################## 당월 입금 필요  #################
########################################################
기준데이터작성후_바로작업 = True 
########################################################
dtype = {"채무자키":str, "계좌키":str, '신용회복키' : str}# 시간정보 무시하고 날짜만 읽기 cf 'datetime64[ns]'
if company == "통합" : company = "솔림"
# 계좌별 진행상황 파일 읽기
if 기준데이터작성후_바로작업 : 
    path_base = join(r"D:\3.자산\전산 dataset", company, basedate)
    credits_ori = 확정미확정
    # 입금(최근1개월) : 채무자,보증인 입금 구분위해 계좌의 최종거래일 쓰지 않고, 실제 입금내역 필요.(회수보고서를 위해서도 어차피 입금내역이 필요)
    deposit_ori = pd.read_excel(functions.키워드로파일명찾기(path_base, "입금조회새창.*당월"), dtype=dtype)
    
else : # 기준데이터 잡업과 별개로 할 때 ~
    ########################################################
    basedate = "250131" 
    company = "대성"
    pkl = True # 신복기준데이터, 계좌만 상관있음
    ########################################################
    
    path_base = join(r"D:\3.자산\전산 dataset", company, basedate)
    if pkl :
        path_base_pkl = join(path_base, "pkl")
        credits_ori = pd.read_pickle(functions.키워드로파일명찾기(폴더=path_base_pkl, 포함키워드="신용회복_기준데이터"))
    else : 
        path_base_pkl = path_base
        credits_ori = pd.read_excel(functions.키워드로파일명찾기(폴더=path_base_pkl, 포함키워드="신용회복_기준데이터"), sheet_name="확정,미확정", dtype=nauri_dtype).fillna("")
        
        # 입금(최근1개월) : 채무자,보증인 입금 구분위해 계좌의 최종거래일 쓰지 않고, 실제 입금내역 필요.(회수보고서를 위해서도 어차피 입금내역이 필요)
    deposit_ori = pd.read_excel(functions.키워드로파일명찾기(path_base, "입금조회새창.*당월"), dtype=dtype)
    
def 말일오류없애기(치환할날:datetime, 변제기일:int):
        """2024-02-31을 2024-02-29로"""
        마지막날 = calendar.monthrange(치환할날.year, 치환할날.month)[1] # 
        if 변제기일 > 마지막날:
            return datetime(치환할날.year, 치환할날.month, 마지막날)
        else:
            return datetime(치환할날.year, 치환할날.month, 변제기일)
                    

def 특정날짜후첫변제일(reference_date:datetime, 변제기일:int) :
    년도 = reference_date.year
    월 = reference_date.month
    날짜 = reference_date.day
    
    해당월말일 = calendar.monthrange(년도, 월)[1]
    if 변제기일 > 해당월말일 : # ex) 변제기일 31, 컷오프 6월 30일이면 첫변제일을 7월 31일이 되어야함
        새변제기일 = 해당월말일 # 31이 30일로 바뀌도록 
    else : 새변제기일 = 변제기일
    
    if 날짜 >= min(변제기일,새변제기일) : # 컷오프 다음날부터 당사 자산
        if 월 == 12 :
            월 = 1
            년도 += 1
        else : 
            월 += 1
            
    첫변제월초일 = datetime(년도, 월, 1) 
    return 말일오류없애기(첫변제월초일, 변제기일) # 7월 31

#### 전처리 및 재시작점

In [29]:
basedate_dt = datetime.strptime(basedate,'%y%m%d')

credit_cols = ["채무자키","계좌키","보증인키","신용회복키","채무구분","신청인명","신청인주민번호","계좌번호","거래일","진행구분","상환구분","신청인진행상태","계좌진행상태내용",
"반영OPB","상환개시일","총상환기간","유예기간","납입회차","연체기간","미납발생일"]
deposit_cols = ["계좌키", "입금일", "입금합계", "입금자구분","입금구분"]

# 복사
credit = credits_ori.copy()[credit_cols]
deposit = deposit_ori.copy()[deposit_cols]

# 신용회복 전처리
    # 확정건만(재조정건도 제외)
credit = credit.query('진행구분=="확정" and 신청인진행상태=="정상이행" and 상환개시일!=""')
    # 날짜타입
credit["상환개시일"] = pd.to_datetime(credit["상환개시일"], errors='coerce')
credit["거래일"] = pd.to_datetime(credit["거래일"], errors='coerce')
credit["미납발생일"] = pd.to_datetime(credit["미납발생일"], errors='coerce')
    # 이달변제일 구하기
credit["변제기일"] = credit["상환개시일"].dt.day
전거래일 = basedate_dt + pd.DateOffset(months=-1) # 거래일은 다음달 월초일 수도 있으니 basedate로 할 것
credit["이달변제예정일"] = credit["변제기일"].apply(lambda x : 특정날짜후첫변제일(전거래일, x))

개시일오류건 = credit[credit["상환개시일"].isna()]
if len(개시일오류건)>0 :
    print("확정건인데 상환개시일 없음")
    display(개시일오류건)
else :
    print("상환개시일 오류 없음")
    # 미납발생건만
credit = credit[~credit["미납발생일"].isna()]

# 입금 전처리
deposit = deposit[deposit["입금구분"]=="신용회복"]
deposit = deposit.drop(columns=["입금구분"])
deposit["입금일"] = pd.to_datetime(deposit["입금일"], errors='coerce')
deposit["입금자구분"] = deposit["입금자구분"].replace({"채무자":"주채무", "보증인":"보증채무"})
deposit = deposit.rename(columns={"입금자구분":"채무구분"})
    # 최근 입금만 남기기
deposit = deposit.sort_values(by="입금일", ascending=False)
deposit = deposit.drop_duplicates(subset=["계좌키","채무구분"], keep='first')

# 입금일 병합
print("병합전신복건",len(credit))
credit = credit.merge(deposit[["계좌키","채무구분","입금일"]], on=["계좌키","채무구분"], how='left')
print("병합후신복건",len(credit))

상환개시일 오류 없음
병합전신복건 4231
병합후신복건 4231


#### 조건 탐색 및 결과저장

In [30]:
# 실효기준미납회차
실효기준미납회차conds = [
    credit["상환개시일"] >= credit["거래일"],
    credit["유예기간"]>=credit["납입회차"],
    credit["납입회차"] - credit["유예기간"] - 18 < 0,
    (credit["납입회차"] - credit["유예기간"] - 12)//6 >= 6, 
    True
]
실효기준미납회차values = [
    1000, # 상환일미도래
    100, # 유예기간
    4,
    10,
    4 + (credit["납입회차"] - credit["유예기간"] - 12)//6
]

credit["실효기준미납회차"] = np.select(실효기준미납회차conds, 실효기준미납회차values)

# 실효후보
    # 이달변제예정일이 입금일로부터 한달이 지난 경우
credit["입금한달초과"] = credit.apply(
    lambda row: True if pd.isna(row["입금일"]) else row["이달변제예정일"] >= row["입금일"] + relativedelta(months=1),
    axis=1
)
    # 3회이상 연체면 실효, 한달 봐줘서 4회로 잡고, 신복처리기간 한 달 추가(+1)
credit["실효조건달성"] = (credit["실효기준미납회차"]+1 < credit["연체기간"]) & credit["입금한달초과"]

# 출력 및 파일저장
실효요청 = credit[credit["실효조건달성"]]
if len(실효요청) > 0 :
    display(실효요청)
    출력_credit_cols = credit_cols + ["실효기준미납회차","입금일"]
    
    functions.save_df_to_excel_underline(실효요청[출력_credit_cols], join(path_base, "_"+company+"_신용회복실효요청할건_" + basedate +".xlsx"), key_columns_no=0)
else :
    print("실효요청할 건이 없음")
    
## 실효임박건은 실효기준미납회차와 연체기간의 차이를 통해 확인할 수 있음

실효요청할 건이 없음


### 채권등급판별 : 변제율
- 진행상태 로그도 받아서 변경전 및 변경후에서 실효이력 있는 거 체크
- 당사전기간 입금건도 체크하는 게 좋겠다.
- 납입회차말고 현재회차 등도 추가

In [ ]:
# # 매입가 파일 갱신된 경우 실해
# 매입가 = pd.read_excel(r"D:\3.자산\매입가.xlsx", dtype={"채무자키":str, "계좌키":str, "계좌번호":str})
# 매입가.to_pickle(r"D:\3.자산\매입가.pkl")

#### 파일읽기

In [ ]:
# 계좌별 진행상황 파일 읽기
########################################################
basedate = "241231" 
company = "솔림"
pkl = True # 신복기준데이터, 계좌만 상관있음
########################################################
path_base = join(r"D:\3.자산\전산 dataset", company, basedate)
if pkl :
    path_base_pkl = join(path_base, "pkl")
else : 
    path_base_pkl = path_base
    
dtype = {"채무자키":str, "계좌키":str, '신용회복키' : str}# 시간정보 무시하고 날짜만 읽기 cf 'datetime64[ns]'
# 파일읽기 : 신복기준데이터, 계좌, 접수번호로그, 합의서체결일로그, 매입가
    # 신복기준데이터
if pkl : 
    credits_ori = pd.read_pickle(functions.키워드로파일명찾기(폴더=path_base_pkl, 포함키워드="신용회복_기준데이터"))
else : 
    credits_ori = pd.read_excel(functions.키워드로파일명찾기(폴더=path_base_pkl, 포함키워드="신용회복_기준데이터"), sheet_name="확정,미확정", dtype=nauri_dtype).fillna("")

    # 계좌
# account = pd.read_excel(r"D:\3.자산\전산 dataset\솔림\240826\_계좌~~.xlsx", dtype={'채무자키':str, '계좌키':str, '타채무자키':str})[["계좌키", "채권구분", "OPB", "컷오프일", "담보제공처"]]
if pkl : 
    account = pd.read_pickle(functions.키워드로파일명찾기(폴더=path_base_pkl, 포함키워드="계좌조회새창"))[["계좌키", "채권구분", "OPB","컷오프일", "담보제공처", "최초연체일"]]
else : 
    account = pd.read_excel(functions.키워드로파일명찾기(폴더=path_base, 포함키워드="계좌조회새창"), dtype=dtype)[["계좌키", "채권구분", "OPB","컷오프일", "담보제공처", "최초연체일"]]

account.rename(columns={"OPB":"최초OPB", "담보제공처":"풀"}, inplace=True)
account["풀"] = account["풀"].str.split("_").str.get(0).fillna("")

#### 기본칼럼에 신용회복키만 추가해서 다운, 파일명 앞에 _추가 ----------------------------------------------------------------------------------------------------
    # 1 접수번호로그 20410653 : 재접수건만 나옴. 접수'일'로그는 재접수아닌 것도 나옴.
접수번호로그_ori = pd.read_excel(functions.키워드로파일명찾기(폴더=path_base, 포함키워드="접수번호로그"), dtype=dtype)[["신용회복키"]]
        # 재접수 아닌 건 삭제 : 신용회복키 고유
접수번호로그_ori = 접수번호로그_ori[접수번호로그_ori.duplicated('신용회복키', keep=False)]
접수번호로그_ori["재접수횟수"] = 접수번호로그_ori.groupby('신용회복키')['신용회복키'].transform('count') - 1
        # 신용회복키 중복값제거
접수번호로그_ori = 접수번호로그_ori.drop_duplicates('신용회복키', keep='first')

    # 2 합의서체결일로그
합의서체결일로그_ori = pd.read_excel(functions.키워드로파일명찾기(폴더=path_base, 포함키워드="합의서체결일로그"), dtype=dtype)[["신용회복키","변경후합의서체결일","변경일"]]
합의서체결일로그_ori["변경일"] = pd.to_datetime(합의서체결일로그_ori["변경일"], errors='coerce')
합의서체결일로그_ori = 합의서체결일로그_ori.rename(columns={'변경후합의서체결일':'합의서체결일로그'})
합의서체결일로그_ori["합의서체결일로그"] = pd.to_datetime(합의서체결일로그_ori["합의서체결일로그"], errors='coerce')

    # 3 감면율로그
감면율로그_ori = pd.read_excel(functions.키워드로파일명찾기(폴더=path_base, 포함키워드="감면율로그"), dtype=dtype)[["신용회복키","변경후감면율", "변경일"]]
감면율로그_ori["변경일"] = pd.to_datetime(감면율로그_ori["변경일"], errors='coerce')

    # 4 신청인진행상태로그 : 실효여부체크용, 변경후는 변경전을 모두 포함(변경후에만 있는 것은 최종실효건). 신복키 반복 횟수는 실효횟수
진행상태로그_ori = pd.read_excel(functions.키워드로파일명찾기(폴더=path_base, 포함키워드="신청인진행상태로그"), dtype=dtype)[["신용회복키","변경후신청인진행상태","변경일"]]
진행상태로그_ori = 진행상태로그_ori.rename(columns={'변경후신청인진행상태':'진행상태로그'})
        # 상태값 실효인 것만 남기기
진행상태로그_ori = 진행상태로그_ori[진행상태로그_ori["진행상태로그"].str.contains('실효')]
진행상태로그_ori["변경일"] = pd.to_datetime(진행상태로그_ori["변경일"], errors='coerce')
진행상태로그_ori["실효횟수"] = 진행상태로그_ori.groupby('신용회복키')['신용회복키'].transform('count')
        # 중복값제거
진행상태로그_ori = 진행상태로그_ori.drop_duplicates('신용회복키', keep='first')

    # 5 상환후잔액로그 : 조정후원금과 차이나는 경우가 있어서(재조정일과 컷오프일이 차이가 날수록) 이왕이면 용량이 커도 상환후잔액으로
상환후잔액로그_ori = pd.read_excel(functions.키워드로파일명찾기(폴더=path_base, 포함키워드="상환후잔액로그"), dtype=dtype)[["신용회복키","변경후상환후잔액"]]
        # 최초건만 남기기
상환후잔액로그_ori = 상환후잔액로그_ori.drop_duplicates('신용회복키', keep='first')
매입후최초상환후잔액_ori = 상환후잔액로그_ori.rename(columns={'변경후상환후잔액':'매입후최초상환후잔액'})

    # 6 입금 : 신용회복 전기간다운(앞선 파일에 추가하는 식으로. 최근 24-12-31까지), 파일명유의, 기본칼럼("계좌키", "입금일", "입금합계", "입금자구분","입금구분"), 최근 6개월 / 전기간
입금_ori = pd.read_excel(functions.키워드로파일명찾기(path_base, "신용회복_입금조회새창"), dtype=dtype)[["계좌키", "입금일", "입금합계", "입금자구분","입금구분"]]
입금_ori["입금일"] = pd.to_datetime(입금_ori["입금일"], errors='coerce')
입금_ori = 입금_ori[입금_ori["입금구분"]=="신용회복"]
입금_ori = 입금_ori.drop(columns=["입금구분"])
입금_ori["입금자구분"] = 입금_ori["입금자구분"].replace({"채무자":"주채무", "보증인":"보증채무"})
#### ------------------------------------------------------------------------------------------------------------------------

    # 매입가
매입가 = pd.read_pickle(r"D:\3.자산\매입가 정리\매입가.pkl")
매입가.dropna(inplace=True)

In [ ]:
# # pkl 저장
# credits_ori.to_pickle(join(path_base_pkl, os.path.splitext(functions.키워드로파일명찾기(폴더=path_base_pkl, 포함키워드="신용회복_기준데이터", 전체경로=False))[0]+".pkl"))
# account.to_pickle(join(path_base_pkl, os.path.splitext(functions.키워드로파일명찾기(폴더=path_base_pkl, 포함키워드="계좌조회새창", 전체경로=False))[0]+".pkl"))

In [9]:
credits = credits_ori.copy()
접수번호로그 = 접수번호로그_ori.copy()
합의서체결일로그 = 합의서체결일로그_ori.copy()
감면율로그 = 감면율로그_ori.copy()
진행상태로그 = 진행상태로그_ori.copy()
매입후최초상환후잔액 = 매입후최초상환후잔액_ori.copy()

입금_전기간 = 입금_ori.copy()
# 6개월
basedate_dt = datetime.strptime(basedate,'%y%m%d')
start_date = basedate_dt - pd.DateOffset(months=6)
입금_6개월 = 입금_ori[(start_date < 입금_ori["입금일"]) & (입금_ori["입금일"] <= basedate_dt)].copy()

#### 파일병합

In [10]:
print("신복계좌수", len(credits))
# 계좌와 병합
credits = credits.merge(account, how='left')
# 풀명칭 변경
credits["풀"] = credits["풀"].replace({'P1': 'P01', 'P2': 'P02', 'P3': 'P03', 'P4': 'P04', 'P5': 'P05', 'P6': 'P06', 'P7': 'P07', 'P8': 'P08', 'P9': 'P09'})
print("계좌병합", len(credits))

# 날짜칼럼 타입변경
credits["컷오프일"] = pd.to_datetime(credits["컷오프일"], errors='coerce')
credits["접수일"] = pd.to_datetime(credits["접수일"], errors='coerce')
credits["상환개시일"] = pd.to_datetime(credits["상환개시일"], errors='coerce')
credits["합의서체결일"] = pd.to_datetime(credits["합의서체결일"], errors='coerce')
credits["최초연체일"] = pd.to_datetime(credits["최초연체일"], errors='coerce')

# 접수번호로그와 병합
credits = credits.merge(접수번호로그, how='left')
credits["재접수횟수"] = credits["재접수횟수"].replace(np.nan,0).astype(int)

print("접수번호", len(credits))



# 합의서체결일로그 : 재조정건 최초합의서체결일 작성
합의서체결일로그 = 합의서체결일로그.merge(credits[["신용회복키", "접수일"]], how='left')
    # 재접수 이전 건은 삭제 : 현재 접수일보다 합의서체결일이 나중인 것, na유의
합의서체결일로그 = 합의서체결일로그[(합의서체결일로그['합의서체결일로그'] > 합의서체결일로그['접수일']) | 합의서체결일로그['접수일'].isna()]
    # 가장 빠른 합의서체결일 남기기
합의서체결일로그 = 합의서체결일로그.drop_duplicates(subset=["신용회복키"], keep='first')
합의서체결일로그 = 합의서체결일로그[["신용회복키", "합의서체결일로그","변경일"]].rename(columns={"합의서체결일로그": "최초합의서체결일"})
    # 병합 : 재조정안한 것 중 합의서체결일과 최초합의서체결일이 다른 경우는 최근에 합의서체결돼서 한곳에는 반영이 안 된 경우뿐
credits = credits.merge(합의서체결일로그[["신용회복키", "최초합의서체결일"]], how='left') # 변경일은 감면율로그와 병합시 사용

print("합의서체결일", len(credits))


# 감면율로그
    # 현재 접수건 중 최초감면율만 남기기 : 최초 합의서체결일 직전
감면율로그 = 감면율로그.merge(합의서체결일로그, on=["신용회복키"], how='left', suffixes=["_감면", "_합의"]) # 최초합의서체결일 na값은 최근 접수건에선 확정전(실효), 확정건이 아니므로 필요없음
감면율로그["날짜차이"] = (감면율로그["변경일_감면"] - 감면율로그["변경일_합의"]).abs() 
감면율로그 = 감면율로그.dropna(subset=["날짜차이"])
감면율로그 = 감면율로그.loc[감면율로그.groupby('신용회복키')['날짜차이'].idxmin()] # inxmin : 최소값을 가지는 index반환
감면율로그 = 감면율로그.rename(columns={"변경후감면율":"최초감면율"})

    # 병합
credits = credits.merge(감면율로그[["신용회복키", "최초감면율"]], how='left')
    # 감면율 변동 없었던 것은 최초감면율로 : 뒤에 연산과정 단순화
credits.loc[credits["최초감면율"].isna(), "최초감면율"] = credits["감면율"]
    # 감면율 변동없었던 것
감면율로그_unique = 감면율로그_ori[~감면율로그_ori.duplicated('신용회복키',keep=False)]

print("감면율", len(credits))


# 진행상태로그 병합
credits = credits.merge(진행상태로그[["신용회복키","실효횟수"]], how='left')

print("진행상태", len(credits))

# 매입후최초상환후잔액 병합
credits = credits.merge(매입후최초상환후잔액[["신용회복키","매입후최초상환후잔액"]], how='left')
print("매입후최초상환후잔액", len(credits))

# 매입가 병합
credits = credits.merge(매입가, how='left') # 매입가 없는 거 있음
print("매입가", len(credits))

# 입금_전기간
입금합계_전기간 = 입금_전기간.groupby(["계좌키", "입금자구분"]).sum('입금합계').reset_index()
입금합계_전기간 = 입금합계_전기간.rename(columns={"입금자구분":"채무구분", "입금합계":"입금합계_전기간"})
# 중복값제거
입금합계_전기간 = 입금합계_전기간.drop_duplicates(["계좌키", "채무구분"], keep='first')
# 병합
credits = credits.merge(입금합계_전기간, on=["계좌키", "채무구분"], how='left')
print("입금합계_전기간", len(credits))

# 입금_6개월
입금합계_6개월 = 입금_6개월.groupby(["계좌키", "입금자구분"]).sum('입금합계').reset_index()
입금합계_6개월 = 입금합계_6개월.rename(columns={"입금자구분":"채무구분", "입금합계":"입금합계_6개월"})
# 중복값제거
입금합계_6개월 = 입금합계_6개월.drop_duplicates(["계좌키", "채무구분"], keep='first')
# 병합
credits = credits.merge(입금합계_6개월, on=["계좌키", "채무구분"], how='left')
print("입금합계_6개월", len(credits))


신복계좌수 11956
계좌병합 11956
접수번호 11956
합의서체결일 11956
감면율 11956
진행상태 11956
매입후최초상환후잔액 11956
매입가 11956
입금합계_전기간 11956
입금합계_6개월 11956


#### 연산 및 저장

In [11]:
# 재조정이전납금금액(조정금액만, 프리이자x) 및 총납입금액은 모두 현재 채권자에게 입금된 금액만을 대상으로 한다!!!

# 전처리
        # 진행구분 세분화
ind = credits[(credits.진행구분=="미확정")&(credits.재조정여부=="Y")].index
credits.loc[ind, "진행구분"] = "재조정중"

        # "", na, 형변환
credits["재조정이전납입금액"] = credits["재조정이전납입금액"].replace('', 0).astype(int)
# credits["재조정이전납입금액"] = credits["재조정이전납입금액"].astype(int)

# 1-1) 최초 조정후원금
        # 당사 최초 조정후원금(상환후잔액) : 매입 후 남은 조정후원금, 금액조정된 경우 차이 있을 수 있음
credits["접수후최초조정후원금"] = np.where(
    ((credits["조정후원금"] == 0)&(credits["조정전원금"] != 0)&(credits["진행구분"] == "확정")), 
    credits["재조정이전납입금액"] + credits["총납입원금"],  # 조정후원금이 0일 때
    credits["재조정이전납입금액"] + credits["조정후원금"]  # 그렇지 않을 때
)
ind = credits[(credits["상환구분"]=="프리")|(credits["원금감면여부"]=="N")|(credits["접수후최초조정후원금"]>credits["조정전원금"])].index
credits.loc[ind, ["접수후최초조정후원금"]] = credits["조정전원금"] # 접수후최초조정후원금 != 조정전원금 왜냐면 매입사 변제


#         # 매입전 최초 조정후원금 : 감면율이 반올림되어있어 약간 오차 있을 수 있음  <---- 재조정이전납입금액이 매입전은 포함하지 않은 줄 알았을 때 작성한 코드
# 감면율 : (조정후합계-비용) / (조정전합계-비용)
# 비용제외조정전합계 = credits["조정전원금"] + credits["조정전이자"] + credits["조정전연체이자"]
# credits["접수최초조정후원금"] = ((100-credits.최초감면율)*비용제외조정전합계/100).round(0)
#                 # 프리채권, 원금감면N, 계산한 최초금액이 - 인 경우 수정
# ind = credits[(credits["상환구분"]=="프리")|(credits["원금감면여부"]=="N")|(credits["접수최초조정후원금"]>credits["조정전원금"])].index
# credits.loc[ind, ["접수최초조정후원금"]] = credits["조정전원금"] # 접수후최초조정후원금 != 조정전원금 왜냐면 매입사 변제

# 1-2) 원금감면율
        # 당사 현재원금감면율은 매입전 변제금 있는 경우 정확하지 않음
credits["최초원금감면율"] = ((credits["조정전원금"]-credits["접수후최초조정후원금"])*100 / credits["조정전원금"]).round(1)
        # 현재원금감면율(기상환액포함)
credits["현재원금감면율"] = ((credits["조정전원금"] - (credits["조정후원금"]+credits["재조정이전납입금액"]))*100 / credits["조정전원금"]).round(1)
                # 프리채권, 원금감면N 수정
credits.loc[(credits["상환구분"]=="프리")|(credits["원금감면여부"]=="N"), ["현재원금감면율"]] = 0
                # 당사현재와 최초원금감면율이 다르지만, 감면율 수정된 적 없는 경우 
ind = credits[(credits["현재원금감면율"]!=credits["최초원금감면율"])
              &(credits["신용회복키"].isin(감면율로그_unique["신용회복키"])) # 감면율변화없음
              &(credits["최초합의서체결일"]<credits["컷오프일"])].index # 컷오프전 확정건(매입전 변제금이 있는 경우 현재원금감면율 계산식이 맞지 않음)
credits.loc[ind,"현재원금감면율"] = credits["최초원금감면율"]


# 2) 접수후 원금상환율 : 접수후 원금입금합계 / 접수일(또는 최초연체일+3) / 접수후기간
credits["총상환원금"] = credits["재조정이전납입금액"]+credits["총납입원금"]

credits["접수후변제기일"] = np.where(credits["접수일"] < credits["최초연체일"], credits["최초연체일"] + pd.DateOffset(months=3), credits["접수일"])
        # 매입후변제기간(월)
credits["접수후변제기간"] = (basedate_dt.year - credits["접수후변제기일"].dt.year) * 12 + (basedate_dt.month - credits["접수후변제기일"].dt.month)
                # 오류건(미확정, 미도래 등) 처리
credits.loc[(credits["접수후변제기간"]<0)|(credits["접수후변제기간"].isna()), "접수후변제기간"] = 0

        # result : 월변제율
credits["접수후월평균납입원금"] = credits["총상환원금"] // credits["접수후변제기간"] # ( / ).astype(int)와 같음, 또는 np.floor( / ).astype(int)
credits["접수후원금상환율"] = (credits["접수후월평균납입원금"]*100 / credits["접수후최초조정후원금"]).round(1)


# 3) 매입후 합계상환율 : 매입후 입금합계_6개월 / 최초조정후합계 / 매입후기간
        # 최초 변제기일 : 최초 합의서체결일 + 1개월
credits["매입후변제기일"] = np.where(credits["최초합의서체결일"]>=credits["컷오프일"], credits["최초합의서체결일"] + pd.DateOffset(months=1), credits["컷오프일"])
        # 매입후변제기간(월)
credits["매입후변제기간"] = (basedate_dt.year - credits["매입후변제기일"].dt.year) * 12 + (basedate_dt.month - credits["매입후변제기일"].dt.month)
                # 오류건(미확정, 미도래 등) 처리
credits.loc[(credits["매입후변제기간"]<0)|(credits["매입후변제기간"].isna()), "매입후변제기간"] = 0

        # result : 월변제율
credits["매입후월평균입금액"] = credits["입금합계_전기간"] // credits["매입후변제기간"]
credits["매입후합계금상환율"] = (credits["매입후월평균입금액"]*100 / credits["매입후최초상환후잔액"]).round(1)

# credits["상환율_접수최초조정후원금"] = (credits["월평균납입원금"]*100 / credits["접수최초조정후원금"]).round(1)
# credits["상환율_최초OPB"] = (credits["월평균납입원금"]*100 / credits["최초OPB"]).round(1)


# 열추가
        # 감면율등급
x = credits["현재원금감면율"]
conds = [x==0, x<=10, x<=30, x<=50, x<70, x<=100]
values = ["0", "10이하", "30이하", "50이하", "70미만", "70이상"]
credits["감면율등급"] = np.select(conds, values, default="")

        # 매입후변제기간등급
x = credits["매입후변제기간"]
conds = [x<=10, x<=40, x<=80, x<=120, x>120]
values = ["10회이하", "40회이하", "80회이하", "120회이하", "120회초과"]
credits["매입후변제기간등급"] = np.select(conds, values, default="")

        # 상환율등급 : 총상환원금 / 조정후원금, 낮을수록 안 좋음
credits["접수후원금상환율등급"] = (credits["접수후원금상환율"]*10).round(0)
credits["매입후합계금상환율등급"] = (credits["매입후합계금상환율"]*10).round(0)

credits.loc[credits["접수후원금상환율등급"]>10, "접수후원금상환율등급"] = 10
credits.loc[credits["매입후합계금상환율등급"]>10, "매입후합계금상환율등급"] = 10


        # 본회차상환액, 유예기간여부
def 본상환액찾기(row) :
    if (row["종료회차1"]=="") or pd.isna(row["종료회차1"]): # 시리즈.isna(), pd.isna(값)
        return 0, 0

    납입회차 = row["납입회차"] # 현재 또는 유예기간 후 본회차금액 찾을 때
    현재회차 = 납입회차 + row["연체기간"] # 유예기간인지 아닌지를 찾을 때
    회차번호=None
    for i in range(1,11) : 
        if 납입회차 <= row[f"종료회차{i}"] :
            회차번호 = i
            break
        # 회차번호 미할당
    if 회차번호 is None :
        return 0, 0
    
        # 유예기간중이면 다음 구간 변제금
    if 납입회차 <= row["유예기간"] : 
        회차번호 += 1
        
    본회차상환액 = row[f"채무조정2월상환액{회차번호}"]
    유예기간여부 = "유예기간" if 현재회차 <= row["유예기간"] else "본기간"
    return 본회차상환액, 유예기간여부

credits[["본회차상환액", "유예기간여부"]] = credits.apply(lambda row : pd.Series(본상환액찾기(row)), axis=1)
        # 최근입금등급 = n개월 입금합계 / 본회차상환액 / n : 중간에 유예면 수치가 이상할 수 있음
##############################################
입금기간 = 6 # 개월수
##############################################
# 최근입금등급 : 최근6개월간 월평균입금액 / 1회차상환액 : 낮을수록 안 좋음
credits["최근입금등급"] = ((credits["입금합계_6개월"] / 입금기간 / credits["본회차상환액"])*10).round(0)
credits.loc[credits["최근입금등급"]>10, "최근입금등급"] = 10

credits["반영/매입"] = (credits["반영OPB"] *100 / credits["매입가"]).round(0)
credits["80%매각시"] = credits["반영/매입"]*0.8
credits["회수율"] = (credits["입금합계_전기간"]*100 / credits["매입가"]).round(1) # 80% 매각시와 더하면 손익계산 가능
credits["매각손익비"] = credits["80%매각시"] + credits["회수율"]


# 후처리
        # 타입재정의
credits[['접수후월평균납입원금', '접수후최초조정후원금', '매입가', "본회차상환액", "입금합계_6개월"]] = \
        credits[['접수후월평균납입원금', '접수후최초조정후원금', '매입가', "본회차상환액", "입금합계_6개월"]].replace([np.inf, -np.inf],0).fillna(0).astype(int)

        # 정렬
credits = credits.sort_values(by=["채무구분", "현재원금감면율", "상환구분",  "진행구분"], ascending=[False,True,False,True])


        # 제외 : 조정전원금 0, 미확정(신규)
credits["구분"] = np.where(((credits["조정전원금"] == 0)|(credits["진행구분"]=="미확정")|(credits["매입후변제기간"]==0)),"미확정등","확정")

credits_확정 = credits.query('구분=="확정"')
credits_미확정 = credits.query('구분!="확정"')


        # 필요칼럼만 남기기
cols = ["풀", "채무자키", "계좌키", "신용회복키", "채무구분", "신청인명", "신청인주민번호", "진행구분", "채권구분", "상환구분",
        "매입후합계금상환율등급", "접수후원금상환율등급", "최근입금등급", "연체기간", "재조정횟수","유예기간여부", # 매각여부 판단열
        "접수후원금상환율", "총상환원금", "접수후변제기일", "접수후최초조정후원금",
        "매입후합계금상환율", "입금합계_전기간", "매입후변제기간", "매입후최초상환후잔액",
        "감면율등급", "현재원금감면율", "최초원금감면율", "감면율","최초감면율",
        "접수일", "합의서체결일", "최초합의서체결일", "컷오프일", 
        "원금감면여부", "상각구분", "최초OPB", "반영OPB", "매입가",
        "재조정이전납입금액", "총납입원금", "총납입이자", "총납입기타채무",
        "재접수횟수", "재조정여부", "재난상환유예", "특례지원", "수정조정여부",
        "총상환기간","유예기간", "납입회차",
        "조정전합계", "조정전원금", "조정전이자", "조정전연체이자", "조정전비용", "조정후합계", "조정후원금", "조정후이자", "조정후연체이자", "조정후비용", 
        "채무조정2월상환액1","본회차상환액", "입금합계_6개월", "반영/매입", "80%매각시", "특별면책", "실효횟수", "회수율", "매각손익비"
        ]



functions.save_df_to_excel_underline(credits_확정[cols], join(path_base, "_"+company+"_신용회복채권등급_확정_" + basedate +".xlsx"), key_columns_no=0)
functions.save_df_to_excel_underline(credits_미확정[cols], join(path_base, "_"+company+"_신용회복채권등급_미확정_" + basedate +".xlsx"), key_columns_no=0)

# 매입후합계금상환율등급 : 접수 후 입금 원금(매입전 포함), 접수후 기간, 낮을 수록 안 좋음
# 최근입금등급 : 입금합계_6개월, 6개월, 낮을 수록 안 좋음
# 0.특별면책 무조건 매각, 1. 매입후합계금상환율등급(3이하, 4이상), 2.최근입금등급, 3.연체기간,재조정횟수,최근입금등급,실효이력
# 핵심 등급 : 매입후합계금상환율등급(주), 접수후원금상환율등급(보조)
#       - 주지표 또는 보조지표가 3이하인 것은 우선 매각대상으로 하되, 입금등급을 비롯한 다른 등급 및 재조정 횟수, 특별면책 등의 기타지표를 통해 최종 결정
# 연체기간 없는 건 일단 보유/유보 (어느 항목에서 뚜렷하게 고등급이면 최소 유보)

### 신용회복 역대 리스트

#### 신용회복 역대 목록작성

In [ ]:
###########################
pkl = True
###########################
path_base = r"C:\Users\DATA\Desktop\신용회복 파일 다운"
dtype = {"채무자키":str, "계좌키":str, '보증인키':str,'신용회복키' : str}

확장자 = "pkl" if pkl else "xlsx"
fn_접수일로그 = functions.키워드로파일명찾기(폴더=path_base, 포함키워드=f"접수일로그.*{확장자}")
fn_신용회복새창 = functions.키워드로파일명찾기(폴더=path_base, 포함키워드=f"신용회복새창.*{확장자}")
# print(fn_접수일로그)
if pkl :
    접수일로그_ori = pd.read_pickle(fn_접수일로그)
    신용회복새창_ori = pd.read_pickle(fn_신용회복새창)
else :
    접수일로그_ori = pd.read_excel(fn_접수일로그, dtype=dtype, parse_dates=["변경후접수일","변경일"])
    신용회복새창_ori = pd.read_excel(fn_신용회복새창, dtype=dtype, parse_dates=["합의서포기일자","접수통지일","확정일", "합의서체결일","실효취소요청일자","실효원상회복일자"])
    
# 신용회복키 빈값은 없는거 확인

In [ ]:
# 접수일로그_ori.to_pickle(fn_접수일로그.split('.')[0]+".pkl")
# 신용회복로그_ori.to_pickle(fn_신용회복새창.split('.')[0]+".pkl")

In [42]:
접수일로그 = 접수일로그_ori.copy()
신용회복 = 신용회복새창_ori.copy()

In [45]:
# 목록 작성
    # 중복제거 : 재접수건은 최근 접수건을 기준으로
신용회복 = 신용회복.sort_values(by=["채무자키","보증인키","접수통지일"], ascending=[True,True,False])
신용회복목록 = 신용회복.drop_duplicates(['채무자키','보증인키'], keep='first')
    # 출력
# 신용회복목록.to_excel(join(path_base,"신용회복목록.xlsx"))
# 파일 열어서 종결,면책 등 제외

#### 파일서치후

In [71]:
# 종결건 삭제한 목록을 새로 읽어도 되고, 삭제전 df를 그냥 사용해도 됨. 파일서치한 목록(종결건 삭제)을 기준으로 할거니까
# 파일서치결과 파일 읽기
fn_서치용목록 = functions.키워드로파일명찾기(폴더=path_base, 포함키워드=f"result_서치용목록")
서치용목록 = pd.read_excel(join(fn_서치용목록), dtype = dtype)

In [72]:
# 접수통지일, 확정일 붙이기
print('병합전 서치용목록', len(서치용목록))
서치용목록 = 서치용목록.merge(신용회복목록[["채무자키","보증인키","접수통지일","확정일"]], on=["채무자키","보증인키"], how='left')
print('병합후 서치용목록', len(서치용목록))

병합전 서치용목록 13787
병합후 서치용목록 13787


In [76]:
# 파일명 작성하기
서치용목록["파일명_고객"] = 서치용목록["채무자키"].str.cat([서치용목록["신청인명"], pd.Series(["고객현황"] * len(서치용목록)), 서치용목록["접수통지일"].dt.strftime("%Y-%m-%d")], sep=" ")
서치용목록["파일명_조정_최초"] = 서치용목록["채무자키"].str.cat([서치용목록["신청인명"], pd.Series(["채무조정안 최초조정"] * len(서치용목록)), 서치용목록["접수통지일"].dt.strftime("%Y")], sep=" ")
서치용목록["파일명_조정_재조"] = 서치용목록["채무자키"].str.cat([서치용목록["신청인명"], pd.Series(["채무조정안 재조정"] * len(서치용목록)), 서치용목록["확정일"].dt.strftime("%Y-%m-%d")], sep=" ", na_rep="")

In [ ]:
# 저장
서치용목록.to_excel(join(path_base, "result_서치용목록_파일명추가.xlsx"), index=False)
# 주민번호와 채권사 첨부하면 됨

### 기준데이터로 계좌별 스케줄 작성

In [20]:
############################################################
path_read = r"D:\3.자산\전산 dataset\솔림\240131\pkl"
fn_read = "240131_신용회복_기준데이터_솔림"
ext = ".pkl"
채무구분 = "주채무" # 주채무, 보증채무, 전체
############################################################
if ext == ".pkl"  :
    credit_ori = pd.read_pickle(join(path_read, fn_read+ext))    
else : 
    credit_ori = pd.read_excel(join(path_base, fn_read+ext), dtype=nauri_dtype)

# 날짜타입
credit_ori["거래일"] = pd.to_datetime(credit_ori["거래일"], errors='coerce')
credit_ori["접수일"] = pd.to_datetime(credit_ori["접수일"], errors='coerce')
credit_ori["상환개시일"] = pd.to_datetime(credit_ori["상환개시일"], errors='coerce')
credit_ori["합의서체결일"] = pd.to_datetime(credit_ori["합의서체결일"], errors='coerce')

In [23]:
# 재시작점 
    # 채무부분 및 확정건(스케줄은 확정건만 가능)
if 채무구분 != "전체" :
    credit = credit_ori[(credit_ori["채무구분"]==채무구분) & (credit_ori["진행구분"]=="확정")].copy()

########### 특정계좌키나 신복키만 선별해서 할 경우
df = pd.read_excel(r"C:\Users\DATA\Desktop\통합 문서1.xlsx", dtype=str)
credit = credit[credit.계좌키.isin(df.계좌키)]
print(len(credit))

1181


#### 스케줄작성

In [24]:
# 5초 정도 느림
def 신용회복_계좌별_스케줄작성(row, 상환일미도래회차추가=False):
    """
    전산 스케줄과 동일한 표 만들기
    이제 초회여부와는 무관. (초회라도 전산매입전상태 인가로 두고, 분납컷오프일 영향을 받도록 함) 2024-11-14
    """
    계좌별스케줄 = []
    
    if pd.isna(row["상환개시일"]): 
        return 계좌별스케줄
    
    # 상환개시일 이전 빈 회차 있는 경우 추가 (부등호 유의)
    if 상환일미도래회차추가 : 
        cnt = -1
        입금반영일 = row['거래일']-pd.DateOffset(days=1)
        while True :
            if row['상환개시일'] + pd.DateOffset(months=cnt) > 입금반영일 : # cnt+1 : 0회차 이하는 미도래회차, 0부터 시작해야 최소회차부터 n개 나옴
                계좌별스케줄.append((row["계좌키"], row["신용회복키"], cnt+1, row['상환개시일'] + pd.DateOffset(months=cnt), 0))
                cnt -= 1
            else :
                break
    
    for i in range(1, 11): # 1~10
        구간시작회차 = int(row[f'시작회차{i}'])
        if 구간시작회차 > 0 :
            구간종료회차 = int(row[f'종료회차{i}'])
            구간변제액 = int(row[f'채무조정2월상환액{i}'])
            for j in range(구간시작회차, 구간종료회차+1) :
                                    # 계좌키        # 신복키       # 회차   # 변제예정일 : j-1유의                    # 구간변제액
                계좌별스케줄.append((row["계좌키"], row["신용회복키"], j, row["상환개시일"]+pd.DateOffset(months=j-1), 구간변제액 ))
        else :
            return 계좌별스케줄
    # 10회차까지 내for문 돈 경우
    return 계좌별스케줄

# 모든 리스트를 하나의 큰 리스트에 수집 후, 마지막에 DataFrame 생성
전체_계좌별스케줄_list = []

######################################
상환일미도래회차추가=True
######################################

for _, row in credit.iterrows(): # apply는 sr나 df를 반환하므로 concat을 해야한다. 리스트와 for문을 활용하고 마지막에 df로 만드는 게 더 효율적
    전체_계좌별스케줄_list.extend(신용회복_계좌별_스케줄작성(row, 상환일미도래회차추가=상환일미도래회차추가)) 

# 최종 DataFrame으로 변환
계좌별스케줄 = pd.DataFrame(전체_계좌별스케줄_list, columns=["계좌키", '신용회복키', '회차', '변제예정일', '변제예정액'])

# 정렬
계좌별스케줄 = 계좌별스케줄.sort_values(["계좌키","회차"])

#### 기간 입금예정액 추출

In [25]:
########################################
n = 12 # 거래일 이후 회차 중 조회할 회차 수
일or월 = "월" # 일:거래일 이후 n회차, 월:(거래일이 16일일지라도)다음달 1일부터 n회차
if 일or월 == "월" : 
    기간시작월 = "2024-02" # 컷오프일 다음월
########################################

credit_sub = credit[['계좌키', '납입회차', '거래일', '상환개시일']].copy()

# 2. credit_sub와 계좌별스케줄을 계좌키를 기준으로 병합
merged = pd.merge(계좌별스케줄, credit_sub, on='계좌키', how='left')

if 일or월 == "일" : 
    # 이후 회차 또는 -회차(미도래)만 (즉 납입회차는 제외하기)
    merged = merged[((merged["납입회차"] < merged["회차"]) | (merged["회차"]<=0))]
else : 
    merged = merged[(기간시작월 <= merged["변제예정일"].dt.to_period("M"))]
    
# n개의 회차만 남기기
    # 최소회차(거래일 후 시작회차) 구하기
merged['최소회차'] = merged.groupby('계좌키')['회차'].transform('min')
    # n개만 남기기
merged = merged[merged["회차"] < merged["최소회차"]+12]

# 1. pivot을 이용하여 계좌키별, 회차별 변제예정액 데이터프레임 생성
pivot_df = merged.pivot(index='계좌키', columns='회차', values='변제예정액')

# pivot_df = pivot_df.rename(columns=[f'변제예정액_{i}' for i in range(1, n+1)])
# # 3. credit과 병합 (left join)
# result = credit.merge(pivot_df, on='계좌키', how='left')

In [26]:
# 계좌키별로 변제예정액만 리스트로 추출
grouped = merged.groupby('계좌키')['변제예정액'].apply(list) # 계좌키, 회차별 정렬 필요. 위에서 했음

# 각 리스트를 max_len 길이로 확장하기 위한 함수
def pad_list(lst, length):
    return lst + [np.nan]*(length - len(lst))

# 각 계좌별 리스트를 패딩 처리(동일한 칼럼수로)
padded = grouped.apply(lambda lst: pad_list(lst, n))

# 3. DataFrame으로 변환하여 칼럼명을 변경하기
#    각 리스트를 Series로 변환하면 인덱스가 0부터 시작하므로, 1부터 시작하는 칼럼명을 만들기
expanded = pd.DataFrame(padded.tolist(), index=padded.index)
expanded.columns = [f'예정액_{i+1}' for i in range(n)]

# 4. credit 데이터프레임과 병합 (계좌키 기준 left join)
result = credit.merge(expanded, left_on='계좌키', right_index=True, how='left')

# 타입변경
for col in expanded.columns:
    result[col] = result[col].astype('Int64')
    
# 소계
# 1~6까지의 합계를 계산
result['예정액합계_6'] = result[[f'예정액_{i}' for i in range(1, 7)]].sum(axis=1)

# 1~12까지의 합계를 계산
result['예정액합계_12'] = result[[f'예정액_{i}' for i in range(1, 13)]].sum(axis=1)

#### 파일저장

In [ ]:
# 필요칼럼만 출력할까? 전체를 출력할까?
#######################################################
path_write = r"C:\Users\DATA\Desktop\신복입금_예정비교"
기준값 = "거래일" if 일or월 == "일" else "거래월"
pool = "P5"
#######################################################
fn_write = pool+"_입금예정액_"+기준값+"이후"+str(n)+"회차.xlsx"

cols = ["채무자키","계좌키","신용회복키","채무구분","신청인명","신청인주민번호","계좌번호","거래일","상환구분","진행구분","신청인진행상태","계좌진행상태내용","반영OPB",
          "확정일","상환개시일","총상환기간","유예기간","납입회차","연체기간","재조정여부",
          "예정액합계_6","예정액합계_12","예정액_1","예정액_2",	"예정액_3",	"예정액_4","예정액_5","예정액_6","예정액_7","예정액_8","예정액_9","예정액_10","예정액_11","예정액_12"]

# 날짜 출력 포맷
result["거래일"] = result["거래일"].dt.strftime("%Y-%m-%d")
result["상환개시일"] = result["상환개시일"].dt.strftime("%Y-%m-%d")

result[cols].to_excel(join(path_write, fn_write), index=False)
# functions.save_df_to_excel_underline(result[cols], join(path_read, fn_write))